In [1]:
import xml.etree.ElementTree as ET
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
from datetime import datetime
from datetime import timedelta

# Functions

In [2]:
# Function to load data to Mysql
def load_MySQL(table_name,new_dataframe):

    # database connection details
    db_username = 'u681559533_AMT'
    db_password = 'MVT2024VTise'
    db_host = '193.203.166.160'
    db_port = '3306'
    db_name = 'u681559533_VT_LF_DB'

    # Create the database engine
    engine = create_engine(f'mysql+pymysql://{db_username}:{db_password}@{db_host}:{db_port}/{db_name}')

    # Load the DataFrame into MySQL table
    new_dataframe.to_sql(name=table_name, con=engine, if_exists='replace', index=False, chunksize=100_000)

    print("DataFrame has been loaded into the database.")

    return None

In [3]:
# Function to create simulation
# The parameters of the function can be change so it can simulate diferent values
def create_simulation(date_start, # Its the date when the simulation start
                      date_finish, # Its the date when the simulation finish
                      mu = 1, # Its the middle value that the distribution its going to take
                      sigma = 1, # Its the variance of the simulation
                      min = 0, # Its the min value that the simulation can take
                      max = 2): # Its the max value that the simulation can take

    # Create date range
    range_dates = pd.date_range(start=date_start, end=date_finish, freq='s')

    # Create DataFrame
    df_dates = pd.DataFrame(range_dates, columns=['timestamp'])

    # Generate the random normal distribution
    random_num = np.random.normal(mu, sigma, size=len(df_dates))

    # Add column to DataFrame
    df_dates['value'] = random_num

    df_dates['value'] = np.where(df_dates['value'] < min, min, df_dates['value'])

    df_dates['value'] = np.where(df_dates['value'] > max, max, df_dates['value'])
    
    return df_dates

In [4]:
# Function to adjust dataframe to XML data structure
def complete_df(df1, df2, deviceName, deviceUuid, component, componentName, componentId, measurementType, dataItemId, name, sequence):
    dft = pd.concat([df1, df2])
    dft['deviceName'] = deviceName
    dft['deviceUuid'] = deviceUuid
    dft['component'] = component
    dft['componentName'] = componentName
    dft['componentId'] = componentId
    dft['measurementType'] = measurementType
    dft['dataItemId'] = dataItemId
    dft['name'] = name
    dft['sequence'] = sequence
    
    dft = dft[['deviceName', 'deviceUuid', 'component', 'componentName', 'componentId', 'measurementType', 'dataItemId', 'timestamp', 'name', 'sequence', 'subType', 'value', 'mantenance']]
    return dft

# Read Data

In [5]:
root = ET.parse('hassur5.xml')

# Prepare a list to hold extracted data
data = []

for device_stream in root.findall('.//{urn:mtconnect.org:MTConnectStreams:1.7}DeviceStream'):
    device_name = device_stream.attrib.get('name')
    device_uuid = device_stream.attrib.get('uuid')
    for component_stream in device_stream.findall('.//{urn:mtconnect.org:MTConnectStreams:1.7}ComponentStream'):
        component = component_stream.attrib.get('component')
        component_name = component_stream.attrib.get('name')
        component_id = component_stream.attrib.get('componentId')
        for samples in component_stream.findall('.//{urn:mtconnect.org:MTConnectStreams:1.7}Samples'):
            for sample in samples:
                # Extract all attributes of the sample
                sample_data = {attr: sample.attrib.get(attr) for attr in sample.attrib}
                sample_data['value'] = sample.text
                sample_data['componentName'] = component_name
                sample_data['componentId'] = component_id
                sample_data['deviceName'] = device_name
                sample_data['deviceUuid'] = device_uuid
                sample_data['component'] = component
                sample_data['measurementType'] = sample.tag.split('}')[1]  # To get 'Amperage', 'Voltage', etc.
                data.append(sample_data)
                
# Convert the list of dictionaries into a DataFrame
df = pd.DataFrame(data)

df = df[['deviceName','deviceUuid','component','componentName','componentId','measurementType','dataItemId', 'timestamp', 'name', 'sequence', 'subType', 'value']].copy()

In [6]:
df[(df['component']=='Rotary')&(df['measurementType']=='Angle')].head()

,deviceName,deviceUuid,component,componentName,componentId,measurementType,dataItemId,timestamp,name,sequence,subType,value
0,VMC-3Axis,HAAS-VF3,Rotary,A,ar,Angle,aposm,2022-01-06T19:57:13.855198Z,aMachine,957858,ACTUAL,94
1,VMC-3Axis,HAAS-VF3,Rotary,A,ar,Angle,aposm,2022-01-06T19:57:16.023847Z,aMachine,957866,ACTUAL,90
2,VMC-3Axis,HAAS-VF3,Rotary,A,ar,Angle,aposm,2022-01-06T20:00:43.07041Z,aMachine,958489,ACTUAL,-6.1077
3,VMC-3Axis,HAAS-VF3,Rotary,A,ar,Angle,aposm,2022-01-06T20:00:44.999016Z,aMachine,958496,ACTUAL,0
4,VMC-3Axis,HAAS-VF3,Rotary,A,ar,Angle,aposm,2022-01-06T20:00:58.644756Z,aMachine,958512,ACTUAL,90


# Simulation

In [7]:
# Parameters
number_of_days = 0
start_date = (datetime.now()- timedelta(days=number_of_days,hours=0,minutes=15)).strftime('%Y-%m-%d %H:%M:%S')
finish_date = (datetime.now()).strftime('%Y-%m-%d %H:%M:%S')

## Angle

In [8]:
# Parameters of machine
dataItemId = 'aposm'
name = 'aMachine'
measurementType = 'Angle'
component = 'Rotary'
componentName = 'A'
componentId = 'ar'
mantenance_date = (datetime.now()- timedelta(days=number_of_days*(2/3))).strftime('%Y-%m-%d %H:%M:%S')
deviceName = 'VMC-3Axis'
deviceUuid = 'HAAS-VF3'
sequence = 22405688

# Simulation
df_targuet = create_simulation(start_date,finish_date, mu = 2, sigma = 1, min = 0, max = 2)
df_targuet['subType'] = 'TARGET'
df_targuet['mantenance'] = 0

df_actual = create_simulation(start_date,finish_date, mu = 1, sigma = 0.5, min = 0, max = 2)
df_actual['subType'] = 'ACTUAL'
df_actual['mantenance'] = np.where(df_actual['timestamp']==mantenance_date, 1, 0)

df_final1 = complete_df(df_targuet, df_actual, deviceName, deviceUuid, component, componentName, componentId, measurementType, dataItemId, name, sequence)

In [9]:
# Parameters of machine
dataItemId = 'aposw'
name = 'aWork'
measurementType = 'Angle'
component = 'Rotary'
componentName = 'A'
componentId = 'ar'
mantenance_date = (datetime.now()- timedelta(days=number_of_days*(2/3))).strftime('%Y-%m-%d %H:%M:%S')
deviceName = 'VMC-3Axis'
deviceUuid = 'HAAS-VF3'
sequence = 22405688

# Simulation
df_targuet = create_simulation(start_date,finish_date, mu = 2, sigma = 1, min = 0, max = 2)
df_targuet['subType'] = 'TARGET'
df_targuet['mantenance'] = 0

df_actual = create_simulation(start_date,finish_date, mu = 1, sigma = 0.5, min = 0, max = 2)
df_actual['subType'] = 'ACTUAL'
df_actual['mantenance'] = np.where(df_actual['timestamp']==mantenance_date, 1, 0)

df_final2 = complete_df(df_targuet, df_actual, deviceName, deviceUuid, component, componentName, componentId, measurementType, dataItemId, name, sequence)

In [10]:
# Create angle dataframe
df_Angle = pd.concat([df_final1, df_final2])

## Rotatory Velocity

In [11]:
# Parameters of machine
dataItemId = 'ss'
name = 'spindleSpeed'
measurementType = 'RotaryVelocity'
component = 'Rotary'
componentName = 'S'
componentId = 'sr'
mantenance_date = (datetime.now()- timedelta(days=number_of_days*(2/3))).strftime('%Y-%m-%d %H:%M:%S')
deviceName = 'VMC-3Axis'
deviceUuid = 'HAAS-VF3'
sequence = 22405688

# Simulation
df_targuet = create_simulation(start_date,finish_date, mu = 20, sigma = 10, min = 0, max = 40)
df_targuet['subType'] = 'TARGET'
df_targuet['mantenance'] = 0

df_actual = create_simulation(start_date,finish_date, mu = 10, sigma = 5, min = 0, max = 30)
df_actual['subType'] = 'ACTUAL'
df_actual['mantenance'] = np.where(df_actual['timestamp']==mantenance_date, 1, 0)

# Create Rotary Velocity dataframe
df_RotaryVelocity = complete_df(df_targuet, df_actual, deviceName, deviceUuid, component, componentName, componentId, measurementType, dataItemId, name, sequence)

## Position

In [12]:
# Parameters of machine
dataItemId = 'xpw'
name = 'xWork'
measurementType = 'Position'
component = 'Linear'
componentName = 'X'
componentId = 'x'
mantenance_date = (datetime.now()- timedelta(days=number_of_days*(2/3))).strftime('%Y-%m-%d %H:%M:%S')
deviceName = 'VMC-3Axis'
deviceUuid = 'HAAS-VF3'
sequence = 22405688

# Simulation
df_targuet = create_simulation(start_date,finish_date, mu = 2, sigma = 1, min = 0, max = 2)
df_targuet['subType'] = 'TARGET'
df_targuet['mantenance'] = 0

df_actual = create_simulation(start_date,finish_date, mu = 1, sigma = 0.5, min = 0, max = 2)
df_actual['subType'] = 'ACTUAL'
df_actual['mantenance'] = np.where(df_actual['timestamp']==mantenance_date, 1, 0)

df_final2 = complete_df(df_targuet, df_actual, deviceName, deviceUuid, component, componentName, componentId, measurementType, dataItemId, name, sequence)

# Parameters of machine
dataItemId = 'xpw'
name = 'xWork'
measurementType = 'Position'
component = 'Linear'
componentName = 'X'
componentId = 'x'
mantenance_date = (datetime.now()- timedelta(days=number_of_days*(2/3))).strftime('%Y-%m-%d %H:%M:%S')
deviceName = 'VMC-3Axis'
deviceUuid = 'HAAS-VF3'
sequence = 22405688

# Simulation
df_targuet = create_simulation(start_date,finish_date, mu = 2, sigma = 1, min = 0, max = 2)
df_targuet['subType'] = 'TARGET'
df_targuet['mantenance'] = 0

df_actual = create_simulation(start_date,finish_date, mu = 1, sigma = 0.5, min = 0, max = 2)
df_actual['subType'] = 'ACTUAL'
df_actual['mantenance'] = np.where(df_actual['timestamp']==mantenance_date, 1, 0)

df_final2 = complete_df(df_targuet, df_actual, deviceName, deviceUuid, component, componentName, componentId, measurementType, dataItemId, name, sequence)

In [13]:
# Parameters of machine
dataItemId = 'ypw'
name = 'yWork'
measurementType = 'Position'
component = 'Linear'
componentName = 'Y'
componentId = 'y'
mantenance_date = (datetime.now()- timedelta(days=number_of_days*(2/3))).strftime('%Y-%m-%d %H:%M:%S')
deviceName = 'VMC-3Axis'
deviceUuid = 'HAAS-VF3'
sequence = 22405688

# Simulation
df_targuet = create_simulation(start_date,finish_date, mu = 2, sigma = 1, min = 0, max = 2)
df_targuet['subType'] = 'TARGET'
df_targuet['mantenance'] = 0

df_actual = create_simulation(start_date,finish_date, mu = 1, sigma = 0.5, min = 0, max = 2)
df_actual['subType'] = 'ACTUAL'
df_actual['mantenance'] = np.where(df_actual['timestamp']==mantenance_date, 1, 0)

df_final3 = complete_df(df_targuet, df_actual, deviceName, deviceUuid, component, componentName, componentId, measurementType, dataItemId, name, sequence)

# Parameters of machine
dataItemId = 'ypw'
name = 'yWork'
measurementType = 'Position'
component = 'Linear'
componentName = 'Y'
componentId = 'y'
mantenance_date = (datetime.now()- timedelta(days=number_of_days*(2/3))).strftime('%Y-%m-%d %H:%M:%S')
deviceName = 'VMC-3Axis'
deviceUuid = 'HAAS-VF3'
sequence = 22405688

# Simulation
df_targuet = create_simulation(start_date,finish_date, mu = 2, sigma = 1, min = 0, max = 2)
df_targuet['subType'] = 'TARGET'
df_targuet['mantenance'] = 0

df_actual = create_simulation(start_date,finish_date, mu = 1, sigma = 0.5, min = 0, max = 2)
df_actual['subType'] = 'ACTUAL'
df_actual['mantenance'] = np.where(df_actual['timestamp']==mantenance_date, 1, 0)

df_final4 = complete_df(df_targuet, df_actual, deviceName, deviceUuid, component, componentName, componentId, measurementType, dataItemId, name, sequence)

In [14]:
# Parameters of machine
dataItemId = 'zpw'
name = 'zWork'
measurementType = 'Position'
component = 'Linear'
componentName = 'Z'
componentId = 'z'
mantenance_date = (datetime.now()- timedelta(days=number_of_days*(2/3))).strftime('%Y-%m-%d %H:%M:%S')
deviceName = 'VMC-3Axis'
deviceUuid = 'HAAS-VF3'
sequence = 22405688

# Simulation
df_targuet = create_simulation(start_date,finish_date, mu = 2, sigma = 1, min = 0, max = 2)
df_targuet['subType'] = 'TARGET'
df_targuet['mantenance'] = 0

df_actual = create_simulation(start_date,finish_date, mu = 1, sigma = 0.5, min = 0, max = 2)
df_actual['subType'] = 'ACTUAL'
df_actual['mantenance'] = np.where(df_actual['timestamp']==mantenance_date, 1, 0)

df_final5 = complete_df(df_targuet, df_actual, deviceName, deviceUuid, component, componentName, componentId, measurementType, dataItemId, name, sequence)

# Parameters of machine
dataItemId = 'zpw'
name = 'zWork'
measurementType = 'Position'
component = 'Linear'
componentName = 'Z'
componentId = 'z'
mantenance_date = (datetime.now()- timedelta(days=number_of_days*(2/3))).strftime('%Y-%m-%d %H:%M:%S')
deviceName = 'VMC-3Axis'
deviceUuid = 'HAAS-VF3'
sequence = 22405688

# Simulation
df_targuet = create_simulation(start_date,finish_date, mu = 2, sigma = 1, min = 0, max = 2)
df_targuet['subType'] = 'TARGET'
df_targuet['mantenance'] = 0

df_actual = create_simulation(start_date,finish_date, mu = 1, sigma = 0.5, min = 0, max = 2)
df_actual['subType'] = 'ACTUAL'
df_actual['mantenance'] = np.where(df_actual['timestamp']==mantenance_date, 1, 0)

df_final6 = complete_df(df_targuet, df_actual, deviceName, deviceUuid, component, componentName, componentId, measurementType, dataItemId, name, sequence)

In [15]:
# Create position dataframe
df_Position = pd.concat([df_final1, df_final2, df_final3, df_final4, df_final5, df_final6])

## Join all tables

In [16]:
# Create final dataframe to load to MySQL
df_final = pd.concat([df_Angle, df_RotaryVelocity, df_Position])

# Load Data

In [17]:
print("Rows to load to MySQL:", len(df_final))
print("Expected time:", (len(df_final)/1_000_000)*1.15, "minutes")

Rows to load to MySQL: 16218
Expected time: 0.0186507 minutes


In [18]:
# Load the data to MySQL
load_MySQL('Hass',df_final)

DataFrame has been loaded into the database.
